<a href="https://colab.research.google.com/github/Vinayak-Sharma12/Types-of-Vector-Stores-or-Databases/blob/main/FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.3 MB/s eta 0:00:00


## **IMPORTING LIBRARIES**

In [7]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


## **LOAD DOCUMENT TEXT**

In [13]:
file_path = "sample_document.txt"  # Replace with your file
with open(file_path, "r", encoding="utf-8") as f:
    document_text = f.read()


## **SPLIT TEXT INTO CHUNKS**

In [15]:
doc_chunks = document_text.split(".")
print(f"Loaded {len(doc_chunks)} chunks")

Loaded 6 chunks


## **LOAD EMBEDDINGS MODEL**

In [16]:
model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **EMBED DOCUMENT CHUNKS**

In [17]:
# Embed document chunks
doc_embeddings = model.encode(doc_chunks, convert_to_numpy=True).astype("float32")
dim = doc_embeddings.shape[1]  # embedding dimension


## **EMBED QUERY**

In [ ]:
#Embedding Query
query = "Explain the main idea of the document"
query_vec = model.encode([query], convert_to_numpy=True).astype("float32")


## **FUNCTIONS TO SAVE AND LOAD INDEXES**

In [19]:
def save_faiss_index(index, path):
    faiss.write_index(index, path)

def load_faiss_index(path):
    return faiss.read_index(path)

## **INDEXING EMBEDDINGS OF DOCUMENT CHUNKS IN FAISS**

### **1. IndexFlatL2(Euclidean)**

In [21]:
index_l2 = faiss.IndexFlatL2(dim)
index_l2.add(doc_embeddings)
save_faiss_index(index_l2, "index_l2.faiss")


In [23]:
#Embedding Query
query = "Explain the main idea of the document"
query_vec = model.encode([query], convert_to_numpy=True).astype("float32")



In [25]:
#Applying Euclidean Similarity search

index_l2_loaded = load_faiss_index("index_l2.faiss")

D, I = index_l2_loaded.search(query_vec, k=3)
print("Results (L2):")
for idx in I[0]:
    print("-", doc_chunks[idx])


Results (L2):
- 
It contains multiple lines of text so we can embed it using SentenceTransformers
and then store those embeddings in a FAISS index
- This is a sample document for FAISS similarity search
- 
Machine learning and AI are transforming the world


### **2. IndexFlatIP(Inner Product)**

In [26]:
#Normalize the Embeddings(Optional if you want to have cosine similarity instead of Inner Product)
faiss.normalize_L2(doc_embeddings)
faiss.normalize_L2(query_vec)

index_ip = faiss.IndexFlatIP(dim)
index_ip.add(doc_embeddings)
save_faiss_index(index_ip, "index_ip.faiss")

# Load and query
index_ip_loaded = load_faiss_index("index_ip.faiss")
D, I = index_ip_loaded.search(query_vec, k=3)
print("Results (IP/Cosine):")
for idx in I[0]:
    print("-", doc_chunks[idx])




Results (IP/Cosine):
- 
It contains multiple lines of text so we can embed it using SentenceTransformers
and then store those embeddings in a FAISS index
- This is a sample document for FAISS similarity search
- 
Machine learning and AI are transforming the world


### **IndexIVFFlat (Approximate search)**

In [27]:
nlist = 5  # number of clusters(to be searched within)

quantizer = faiss.IndexFlatL2(dim)  #Tells how to cluster

index_ivf = faiss.IndexIVFFlat(quantizer, dim, nlist, faiss.METRIC_L2)

# Must train before adding vectors
index_ivf.train(doc_embeddings)
index_ivf.add(doc_embeddings)
index_ivf.nprobe = 3  # search 3 clusters

save_faiss_index(index_ivf, "index_ivf.faiss")

# Load and query
index_ivf_loaded = load_faiss_index("index_ivf.faiss")
D, I = index_ivf_loaded.search(query_vec, k=3)
print("Results (IVF):")
for idx in I[0]:
    print("-", doc_chunks[idx])


Results (IVF):
- 
It contains multiple lines of text so we can embed it using SentenceTransformers
and then store those embeddings in a FAISS index
- This is a sample document for FAISS similarity search
- 

The quick brown fox jumps over the lazy dog
